In [1]:
# Imports
import cv2
import glob
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import numpy as np

from skimage.feature import hog
from skimage import color, exposure
from sklearn.calibration import CalibratedClassifierCV
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

import features as F


In [2]:
# Load Data and Feature engineering.

def LoadData(imagePath):
    images = glob.glob(imagePath)
    data = []

    for image in images:
        data.append(image)
    return data


carPath = "./data/vehicles/*/*.png"
notCarPath = "./data/non-vehicles/*/*.png"
cars = LoadData(carPath)
notCars = LoadData(notCarPath)

# Prepare Features.
print("Number of vehicles images: ", len(cars))
print("Number of non-vehicles images: ", len(notCars))

f = F.Features()

car_features = f.extract_features(cars)
notcar_features = f.extract_features(notCars)



Number of vehicles images:  8792
Number of non-vehicles images:  8968


In [3]:
# Train and Save Classifier

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC(C=1.0)
clf = CalibratedClassifierCV(svc)
# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))
print('Train Accuracy of SVC = ', round(clf.score(X_train, y_train), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', clf.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
print('with probability: ', clf.predict_proba(X_test[0:n_predict]))

t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
# Save model & scaler
print("Saving models...")
joblib.dump(svc, './models/classifier.pkl')
joblib.dump(clf, './models/calibrated.pkl')
joblib.dump(X_scaler, './models/scaler.pkl')
print("Done!")

Feature vector length: 8460
70.23 Seconds to train SVC...
Test Accuracy of SVC =  0.9792
Train Accuracy of SVC =  1.0
My SVC predicts:  [ 0.  1.  0.  0.  0.  0.  1.  1.  1.  0.]
For these 10 labels:  [ 0.  1.  0.  0.  0.  1.  1.  1.  1.  0.]
with probability:  [[  9.90681532e-01   9.31846789e-03]
 [  2.89918904e-04   9.99710081e-01]
 [  9.99996731e-01   3.26929411e-06]
 [  9.41452600e-01   5.85473998e-02]
 [  9.97634850e-01   2.36515013e-03]
 [  8.10450463e-01   1.89549537e-01]
 [  2.21678531e-02   9.77832147e-01]
 [  7.89584568e-05   9.99921042e-01]
 [  1.45485759e-03   9.98545142e-01]
 [  9.42698196e-01   5.73018043e-02]]
0.00518 Seconds to predict 10 labels with SVC
Saving models...
Done!


In [5]:
# Load Models

# Load the classifier model
svc = joblib.load('./models/classifier.pkl')
clf = joblib.load('./models/calibrated.pkl')

# Load the standard scalar model
X_scaler = joblib.load('./models/scaler.pkl')

In [13]:
# Utility functions for feature engineering    

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows


# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars_old(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                
    return draw_img


# Define a single function that can extract features and make predictions
def find_cars(img, windows):
    img = img.astype(np.float32)/255
    possible_cars = []
    f = F.Features()

    # Iterate over all windows in the list
    for w in windows:
        # Extract the test window from original image
        sub_image = img[w[0][1]:w[1][1], w[0][0]:w[1][0]]
        test_img = cv2.resize(sub_image, (64, 64))
        # Extract features for that window 
        features = f.extract_features_image(test_img)
        # Scale extracted features to be fed to classifier
        test_features = X_scaler.transform(np.array(features).reshape(1, -1))
        # Predict using your classifier
        prediction = clf.predict(test_features)
        # If positive (prediction == 1) then save the window
        if prediction == 1:
            # Confidence score
            confidence_score = clf.predict_proba(scaled_X)
            print(confidence_score)
            possible_cars.append(w)
    # Return windows for positive detections
    return possible_cars


In [14]:
import pipeline as p

def process_image(img):

    windows = p.getWindows(img)
    on_windows = find_cars(img, windows)
    final_img = draw_boxes(img, on_windows)
#     print("all:", len(windows))
    return final_img


In [15]:
# Test images

image_path = "./test_images/*.jpg"
images = glob.glob(image_path)

fig = plt.figure()
fig.set_size_inches(50,50)

total_image_count = len(images)
image_count = 1

for fname in images:
    print("Testing image {}".format(fname))

    img = mpimg.imread(fname)
    marked_img = process_image(img)

    ax = fig.add_subplot(total_image_count, 2, image_count)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Original Image ', fontsize=30)
    image_count += 1

    ax = fig.add_subplot(total_image_count, 2, image_count)
    ax.imshow(marked_img)
    ax.axis('off')
    ax.set_title('Marked Image ', fontsize=30)
    image_count += 1

plt.show()


Testing image ./test_images/test1.jpg
[[  2.79957399e-03   9.97200426e-01]
 [  7.74053569e-02   9.22594643e-01]
 [  1.35059359e-04   9.99864941e-01]
 ..., 
 [  9.99973937e-01   2.60625590e-05]
 [  9.99939489e-01   6.05113460e-05]
 [  9.99995487e-01   4.51306963e-06]]
[[  2.79957399e-03   9.97200426e-01]
 [  7.74053569e-02   9.22594643e-01]
 [  1.35059359e-04   9.99864941e-01]
 ..., 
 [  9.99973937e-01   2.60625590e-05]
 [  9.99939489e-01   6.05113460e-05]
 [  9.99995487e-01   4.51306963e-06]]
[[  2.79957399e-03   9.97200426e-01]
 [  7.74053569e-02   9.22594643e-01]
 [  1.35059359e-04   9.99864941e-01]
 ..., 
 [  9.99973937e-01   2.60625590e-05]
 [  9.99939489e-01   6.05113460e-05]
 [  9.99995487e-01   4.51306963e-06]]
[[  2.79957399e-03   9.97200426e-01]
 [  7.74053569e-02   9.22594643e-01]
 [  1.35059359e-04   9.99864941e-01]
 ..., 
 [  9.99973937e-01   2.60625590e-05]
 [  9.99939489e-01   6.05113460e-05]
 [  9.99995487e-01   4.51306963e-06]]
[[  2.79957399e-03   9.97200426e-01]
 [  7

SystemError: new style getargs format but argument is not a tuple

In [4]:
# Test Video
clip = VideoFileClip("./project_video.mp4")
output_video = "./project_video_marked.mp4"
output_clip = clip.fl_image(process_image)
output_clip.write_videofile(output_video, audio=False)


l: 210
m: 1067
s: 2842
all: 4119
[MoviePy] >>>> Building video ./project_video_marked.mp4
[MoviePy] Writing video ./project_video_marked.mp4


  0%|          | 5/1261 [00:00<00:25, 49.14it/s]

l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067

  1%|          | 15/1261 [00:00<00:21, 57.64it/s]


s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l:

...

l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m:

100%|█████████▉| 1259/1261 [00:26<00:00, 45.23it/s]

 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 4119
l: 210
m: 1067
s: 2842
all: 

100%|█████████▉| 1260/1261 [00:26<00:00, 47.88it/s]


4119
[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_marked.mp4 

